In [1]:
import reservoirflow as rf
import numpy as np

In [2]:
type(rf.scalers.MinMax((1, 1)))

reservoirflow.scalers.minmax.MinMax

In [3]:
def create_model_example_7_1():
    grid = rf.grids.Cartesian(
        nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
    )
    fluid = rf.fluids.SinglePhase(mu=0.5, B=1, dtype="double")
    model = rf.models.Numerical(grid, fluid, dtype="double", verbose=False)
    model.set_well(id=4, q=-600, s=1.5, r=3.5)
    model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
    return model


def create_model():
    grid = rf.grids.Cartesian(
        nx=3,
        ny=1,
        nz=1,
        dx=300,
        dy=350,
        dz=20,
        phi=0.27,
        kx=1,
        ky=1,
        kz=0.1,
        comp=1 * 10**-6,
        dtype="double",
    )
    fluid = rf.fluids.SinglePhase(
        mu=0.5, B=1, rho=50, comp=1 * 10**-5, dtype="double"
    )
    model = rf.models.Numerical(
        grid, fluid, pi=6000, dt=5, start_date="10.10.2018", dtype="double"
    )

    n_samples = 1
    cells_id = model.grid.get_cells_id(False, True)[-1].flatten()
    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=-300, pwf=100, s=1.5, r=3.5)

    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=100, s=0, r=3.5)

    return model


model = create_model()
model.run(10, isolver="cgs")

[info] Simulation run started: 10 timesteps.


[step]: 10 - : 100%|██████████| 10/10 [00:00<00:00, 304.71steps/s]

[info] Simulation run of 10 steps finished in 0.04 seconds.
[info] Material Balance Error: 2.55351295663786e-15.


In [4]:
arr = model.grid.get_cells_center(True, False)
print(arr.shape)
print(arr.min(axis=0))
arr

(5, 3)
[150. 175.  10.]


array([[ 150.,  175.,   10.],
       [ 450.,  175.,   10.],
       [ 750.,  175.,   10.],
       [1050.,  175.,   10.],
       [1350.,  175.,   10.]])

In [5]:
np.count_nonzero(np.array([1, 2, 0, 0]) == 0)

2

In [6]:
scaler = rf.scalers.MinMax((-1, 1)).fit(arr, axis=0)
scaler.transform(arr)

c:\users\abugrzka\projects\openresim\reservoirflow\scalers\minmax.py:35: RuntimeWarning: invalid value encountered in true_divide
  self.vmax - self.vmin


array([[-1. ,  nan,  nan],
       [-0.5,  nan,  nan],
       [ 0. ,  nan,  nan],
       [ 0.5,  nan,  nan],
       [ 1. ,  nan,  nan]])

In [20]:
t = np.arange(0, (model.tstep + 1) * model.dt, model.dt)
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [21]:
t

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

In [22]:
t, x = model.get_domain(False, True)
t, x

(array([[ 0],
        [ 5],
        [10],
        [15],
        [20],
        [25],
        [30],
        [35],
        [40],
        [45],
        [50]]),
 array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]))

In [23]:
rf.scalers.MinMax((-1, 1)).fit_transform(x, axis=0)

array([[-1. ],
       [-0.5],
       [ 0. ],
       [ 0.5],
       [ 1. ]])

In [25]:
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

array([[0. ],
       [0.1],
       [0.2],
       [0.3],
       [0.4],
       [0.5],
       [0.6],
       [0.7],
       [0.8],
       [0.9],
       [1. ]])

In [26]:
model.get_centers(False)

(array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]),
 ['x'])

In [28]:
config = {"time": ["MinMax", (0, 1)], "space": ["minMax"]}

In [29]:
model.set_scalers({"time": ["minmax", (-10, 10)], "rate": ["minmax", None]})
model.scalers_dict

{'time': ['MinMax', (-10, 10)],
 'space': ['MinMax', (-1, 1)],
 'pressure': ['MinMax', (-1, 1)],
 'rate': [None, None]}

In [30]:
model.update_scalers(False)
model.pressures_scaler.transform(model.pressures[1])
# model.space_scaler.transform(model.get_centers(False)[0])

array([-1.        , -0.69223375, -0.94898548, -0.9902352 , -1.        ])

In [31]:
model.get_df(
    scale=True,
    melt=False,
    boundary=True,
    units=True,
    columns=["time", "cells", "wells"],
)

,Time [Scaled],Q1 [Scaled],P0 [Scaled],P1 [Scaled],P2 [Scaled],P3 [Scaled],P4 [Scaled],Qw1 [Scaled],Pwf1 [Scaled]
Step,,,,,,,,,
0,-10.0,0.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.0,-1.000000
1,-8.0,100.0,-1.0,-0.692234,-0.948985,-0.990235,-1.0,100.0,0.868610
2,-6.0,100.0,-1.0,-0.427903,-0.868141,-0.966865,-1.0,100.0,1.132941
3,-4.0,100.0,-1.0,-0.195355,-0.769854,-0.929155,-1.0,100.0,1.365489
4,-2.0,100.0,-1.0,0.013446,-0.661367,-0.877897,-1.0,100.0,1.574290
5,0.0,100.0,-1.0,0.204187,-0.546916,-0.814543,-1.0,100.0,1.765031
6,2.0,100.0,-1.0,0.380993,-0.428978,-0.740741,-1.0,100.0,1.941837
7,4.0,100.0,-1.0,0.546921,-0.309001,-0.658101,-1.0,100.0,2.107765
8,6.0,100.0,-1.0,0.704282,-0.187832,-0.568086,-1.0,100.0,2.265126
